In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 10
    self.test_batch = 1000
    #self.global_epochs = 300
    self.local_epochs = 100
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 100
    self.worker_num = 20
    #self.sample_num = 20
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
lr = 0

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for i in range(len(dataset)):
      self.data.append(dataset[i][0][0])
      self.target.append(dataset[i][1][0])

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_shakespeare.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_shakespeare.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[28, 6, 70, 62, 57, 35, 26, 139, 22, 108, 8, 7, 23, 55, 59, 129, 50, 107, 56, 114]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class RNN1(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN1, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN2(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN2, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN3(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN3, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=3, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output
    
class RNN4(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN4, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=4, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

In [15]:
class Server():
  def __init__(self):
    self.model = RNN2()

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_worker_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_worker_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_model(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.model = copy.deepcopy(self.model)
      worker.model = worker.model.to(args.device)

  def aggregate_model(self,workers):   
    new_params = OrderedDict()
    for i,worker in enumerate(workers):
      worker_state = worker.model.state_dict()
      for key in worker_state.keys():
        if i==0:
          new_params[key] = worker_state[key]*worker.aggregation_weight
        else:
          new_params[key] += worker_state[key]*worker.aggregation_weight
      worker.model = worker.model.to('cpu')
      del worker.model
    self.model.load_state_dict(new_params)

In [16]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.trainloader = torch.utils.data.DataLoader(trainset,batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.model = None
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    self.aggregation_weight = None

  def local_train(self):
    acc_valid,loss_valid = train(self.model,args.criterion,self.trainloader,self.valloader,args.local_epochs,partience=args.partience,early_stop=True)
    return acc_valid[-1],loss_valid[-1]

In [17]:
def train(model,criterion,trainloader,valloader,epochs,partience=0,early_stop=False):
  if early_stop:
    early_stopping = Early_Stopping(partience)

  acc_valid = []
  loss_valid = []
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    model.train()
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()
      
    model.eval()
    for (data,labels) in valloader:
      count += len(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      
    print('Epoch:{}  accuracy:{}  loss:{}'.format(epoch+1,100.0*correct/count,running_loss/len(valloader)))
    acc_valid.append(100.0*correct/count)
    loss_valid.append(running_loss/len(valloader))
    if early_stop:
      if early_stopping.validate(running_loss):
        print('Early Stop')
        return acc_valid,loss_valid

  return acc_valid,loss_valid

In [18]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [19]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [20]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()#開始時刻

for i,worker in enumerate(workers):
    print('Worker{} start'.format(i+1))
    worker.model = RNN2()
    worker.model = worker.model.to(args.device)
    acc_tmp,loss_tmp = worker.local_train()
    acc_valid.append(acc_tmp)
    loss_valid.append(loss_tmp)
    worker.model = worker.model.to('cpu')
    
end = time.time()#終了時刻

Worker1 start
Epoch:1  accuracy:18.61604409063074  loss:3.1486911508772106
Epoch:2  accuracy:18.61604409063074  loss:3.118537876341078
Epoch:3  accuracy:18.61604409063074  loss:3.115110264884101
Epoch:4  accuracy:18.61604409063074  loss:3.1076209810045032
Epoch:5  accuracy:18.61604409063074  loss:3.105384190877279
Epoch:6  accuracy:18.61604409063074  loss:3.1215279367234974
Epoch:7  accuracy:18.61604409063074  loss:3.0868136088053384
Epoch:8  accuracy:18.444580526638088  loss:3.064385096232096
Epoch:9  accuracy:20.25719534598898  loss:3.0219382445017495
Epoch:10  accuracy:20.943049601959583  loss:2.983137845993042
Epoch:11  accuracy:21.249234537660747  loss:2.9525361326005726
Epoch:13  accuracy:22.841396203306797  loss:2.8719292216830783
Epoch:14  accuracy:23.306797305572566  loss:2.827807823816935
Epoch:15  accuracy:24.262094304960197  loss:2.7795510821872287
Epoch:16  accuracy:25.254133496631965  loss:2.7280083762274847
Epoch:17  accuracy:25.229638701775873  loss:2.6937519444359674
E

Epoch:35  accuracy:18.33810888252149  loss:3.2248222827911377
Epoch:36  accuracy:18.33810888252149  loss:3.2231528759002686
Epoch:37  accuracy:18.33810888252149  loss:3.221156120300293
Epoch:38  accuracy:18.33810888252149  loss:3.2223708629608154
Epoch:39  accuracy:18.33810888252149  loss:3.220662832260132
Epoch:40  accuracy:18.33810888252149  loss:3.2260687351226807
Epoch:41  accuracy:18.33810888252149  loss:3.220625877380371
Epoch:42  accuracy:18.33810888252149  loss:3.2190043926239014
Epoch:43  accuracy:18.33810888252149  loss:3.222245693206787
Epoch:44  accuracy:18.33810888252149  loss:3.2232275009155273
Epoch:45  accuracy:18.33810888252149  loss:3.216989278793335
Epoch:46  accuracy:18.33810888252149  loss:3.222698211669922
Epoch:47  accuracy:18.33810888252149  loss:3.2185208797454834
Epoch:48  accuracy:18.33810888252149  loss:3.21962571144104
Epoch:49  accuracy:18.33810888252149  loss:3.2197136878967285
Epoch:50  accuracy:18.33810888252149  loss:3.2169864177703857
Epoch:51  accura

Epoch:68  accuracy:18.867924528301888  loss:3.173130512237549
Epoch:69  accuracy:18.867924528301888  loss:3.1661694049835205
Epoch:70  accuracy:18.867924528301888  loss:3.16613507270813
Epoch:71  accuracy:18.867924528301888  loss:3.168424606323242
Epoch:72  accuracy:18.867924528301888  loss:3.1577422618865967
Epoch:73  accuracy:18.867924528301888  loss:3.1423580646514893
Epoch:74  accuracy:18.867924528301888  loss:3.13348126411438
Epoch:75  accuracy:18.867924528301888  loss:3.126765012741089
Epoch:76  accuracy:18.867924528301888  loss:3.125476121902466
Epoch:77  accuracy:18.867924528301888  loss:3.125183582305908
Epoch:78  accuracy:18.867924528301888  loss:3.1207833290100098
Epoch:79  accuracy:18.867924528301888  loss:3.1249356269836426
Epoch:80  accuracy:18.867924528301888  loss:3.116079330444336
Epoch:81  accuracy:18.867924528301888  loss:3.1086950302124023
Epoch:82  accuracy:18.867924528301888  loss:3.1066622734069824
Epoch:83  accuracy:18.867924528301888  loss:3.104383945465088
Epo

Epoch:1  accuracy:2.6315789473684212  loss:4.491479396820068
Epoch:2  accuracy:2.6315789473684212  loss:4.486478328704834
Epoch:3  accuracy:2.6315789473684212  loss:4.4809722900390625
Epoch:4  accuracy:2.6315789473684212  loss:4.474759578704834
Epoch:5  accuracy:2.6315789473684212  loss:4.468466281890869
Epoch:6  accuracy:2.6315789473684212  loss:4.462224006652832
Epoch:7  accuracy:10.526315789473685  loss:4.455883026123047
Epoch:8  accuracy:10.526315789473685  loss:4.44955587387085
Epoch:9  accuracy:10.526315789473685  loss:4.443349838256836
Epoch:10  accuracy:10.526315789473685  loss:4.436854362487793
Epoch:11  accuracy:10.526315789473685  loss:4.430279731750488
Epoch:12  accuracy:10.526315789473685  loss:4.423807621002197
Epoch:13  accuracy:10.526315789473685  loss:4.417612075805664
Epoch:14  accuracy:10.526315789473685  loss:4.411118030548096
Epoch:15  accuracy:10.526315789473685  loss:4.4045939445495605
Epoch:16  accuracy:10.526315789473685  loss:4.398208141326904
Epoch:17  accura

Epoch:35  accuracy:16.666666666666668  loss:4.354092121124268
Epoch:36  accuracy:16.666666666666668  loss:4.349672794342041
Epoch:37  accuracy:16.666666666666668  loss:4.345517158508301
Epoch:38  accuracy:16.666666666666668  loss:4.341277122497559
Epoch:39  accuracy:16.666666666666668  loss:4.336878776550293
Epoch:40  accuracy:16.666666666666668  loss:4.332719802856445
Epoch:41  accuracy:16.666666666666668  loss:4.328542709350586
Epoch:42  accuracy:16.666666666666668  loss:4.324111461639404
Epoch:43  accuracy:16.666666666666668  loss:4.319706439971924
Epoch:44  accuracy:16.666666666666668  loss:4.315385341644287
Epoch:45  accuracy:16.666666666666668  loss:4.3110737800598145
Epoch:46  accuracy:16.666666666666668  loss:4.306848049163818
Epoch:47  accuracy:16.666666666666668  loss:4.302502632141113
Epoch:48  accuracy:16.666666666666668  loss:4.297891616821289
Epoch:49  accuracy:16.666666666666668  loss:4.2928595542907715
Epoch:50  accuracy:16.666666666666668  loss:4.288119792938232
Epoch:

...

Epoch:86  accuracy:34.56472855766142  loss:2.373402774333954
Epoch:87  accuracy:35.0787022165114  loss:2.3701154589653015
Epoch:88  accuracy:35.0787022165114  loss:2.3643566966056824
Epoch:89  accuracy:34.532605203983294  loss:2.368120014667511
Epoch:90  accuracy:34.918085448120785  loss:2.3765076398849487
Epoch:91  accuracy:34.59685191133954  loss:2.3535879254341125
Epoch:92  accuracy:35.43205910697077  loss:2.3551486134529114
Epoch:93  accuracy:35.52842916800514  loss:2.355376362800598
Epoch:94  accuracy:35.27144233858015  loss:2.3547325134277344
Epoch:95  accuracy:35.94603276582075  loss:2.347601532936096
Epoch:96  accuracy:36.042402826855124  loss:2.345323324203491
Epoch:97  accuracy:36.010279473177  loss:2.3424474596977234
Epoch:98  accuracy:35.43205910697077  loss:2.3463568091392517
Epoch:99  accuracy:36.042402826855124  loss:2.3311917185783386
Epoch:100  accuracy:36.65274654673948  loss:2.3147172927856445
Worker14 start
Epoch:1  accuracy:16.209476309226932  loss:4.42668247222900

Epoch:23  accuracy:10.0  loss:4.484407901763916
Epoch:24  accuracy:10.0  loss:4.482901573181152
Epoch:25  accuracy:10.0  loss:4.481595039367676
Epoch:26  accuracy:10.0  loss:4.480316638946533
Epoch:27  accuracy:10.0  loss:4.478793144226074
Epoch:28  accuracy:10.0  loss:4.4772257804870605
Epoch:29  accuracy:10.0  loss:4.47575569152832
Epoch:30  accuracy:10.0  loss:4.474245071411133
Epoch:31  accuracy:10.0  loss:4.47292947769165
Epoch:32  accuracy:10.0  loss:4.471663475036621
Epoch:33  accuracy:10.0  loss:4.470297813415527
Epoch:34  accuracy:10.0  loss:4.469080924987793
Epoch:35  accuracy:10.0  loss:4.4680094718933105
Epoch:36  accuracy:10.0  loss:4.466901779174805
Epoch:37  accuracy:10.0  loss:4.4655070304870605
Epoch:38  accuracy:10.0  loss:4.464034080505371
Epoch:39  accuracy:10.0  loss:4.462754249572754
Epoch:40  accuracy:0.0  loss:4.461661338806152
Epoch:41  accuracy:0.0  loss:4.460541725158691
Epoch:42  accuracy:0.0  loss:4.459142208099365
Epoch:43  accuracy:0.0  loss:4.45754814147

Epoch:73  accuracy:32.896117523609654  loss:2.4004116654396057
Epoch:74  accuracy:32.712486883525706  loss:2.3838558197021484
Epoch:75  accuracy:33.7093389296957  loss:2.3868849873542786
Epoch:76  accuracy:33.7093389296957  loss:2.3659104108810425
Epoch:77  accuracy:33.31584470094439  loss:2.3654114603996277
Epoch:78  accuracy:32.922350472193074  loss:2.366914749145508
Epoch:79  accuracy:34.81112277019937  loss:2.343853235244751
Epoch:80  accuracy:34.73242392444911  loss:2.336832821369171
Epoch:81  accuracy:34.70619097586569  loss:2.3336893916130066
Epoch:82  accuracy:34.28646379853095  loss:2.3425930738449097
Epoch:83  accuracy:33.971668415529905  loss:2.3310571908950806
Epoch:84  accuracy:34.81112277019937  loss:2.323542892932892
Epoch:85  accuracy:35.204616998950684  loss:2.305777847766876
Epoch:86  accuracy:35.75550891920252  loss:2.3005893230438232
Epoch:87  accuracy:35.44071353620147  loss:2.3013240098953247
Epoch:88  accuracy:35.99160545645331  loss:2.2854475378990173
Epoch:89  

Epoch:6  accuracy:25.0  loss:4.396905899047852
Epoch:7  accuracy:25.0  loss:4.376923561096191
Epoch:8  accuracy:25.0  loss:4.357165336608887
Epoch:9  accuracy:25.0  loss:4.337282657623291
Epoch:10  accuracy:25.0  loss:4.315758228302002
Epoch:11  accuracy:25.0  loss:4.293973922729492
Epoch:12  accuracy:25.0  loss:4.2723846435546875
Epoch:13  accuracy:25.0  loss:4.250145435333252
Epoch:14  accuracy:25.0  loss:4.228387832641602
Epoch:15  accuracy:25.0  loss:4.20399284362793
Epoch:16  accuracy:25.0  loss:4.178944110870361
Epoch:17  accuracy:25.0  loss:4.15255069732666
Epoch:18  accuracy:25.0  loss:4.1270880699157715
Epoch:19  accuracy:25.0  loss:4.098607063293457
Epoch:20  accuracy:25.0  loss:4.067458152770996
Epoch:21  accuracy:25.0  loss:4.035757064819336
Epoch:22  accuracy:25.0  loss:4.001965522766113
Epoch:23  accuracy:25.0  loss:3.966278076171875
Epoch:24  accuracy:25.0  loss:3.9295668601989746
Epoch:25  accuracy:25.0  loss:3.8898353576660156
Epoch:26  accuracy:25.0  loss:3.8459486961

Epoch:59  accuracy:27.52704214845207  loss:2.656424045562744
Epoch:60  accuracy:27.63894069377098  loss:2.648080587387085
Epoch:61  accuracy:27.265945542707946  loss:2.639937400817871
Epoch:62  accuracy:28.384930995897054  loss:2.626230239868164
Epoch:63  accuracy:28.944423722491607  loss:2.6144770781199136
Epoch:64  accuracy:28.422230511003356  loss:2.6056386629740396
Epoch:65  accuracy:28.907124207385305  loss:2.5930046240488687
Epoch:66  accuracy:28.8325251771727  loss:2.583352486292521
Epoch:67  accuracy:29.019022752704213  loss:2.5825725396474204
Epoch:68  accuracy:29.39201790376725  loss:2.568549315134684
Epoch:69  accuracy:30.21260723610593  loss:2.5641608238220215
Epoch:70  accuracy:30.32450578142484  loss:2.5510810216267905
Epoch:71  accuracy:30.43640432674375  loss:2.5387128988901773
Epoch:72  accuracy:30.21260723610593  loss:2.5403590202331543
Epoch:73  accuracy:30.361805296531145  loss:2.52181339263916
Epoch:74  accuracy:30.958597538232002  loss:2.521624485651652
Epoch:75  

In [21]:
print('学習時間：{}秒'.format(end-start))#終了時刻-開始時刻でかかった時間

学習時間：24766.437942028046秒


In [22]:
acc_test = []
loss_test = []

start = time.time()#開始時刻

for i,worker in enumerate(workers):
    worker.model = worker.model.to(args.device)
    acc_tmp,loss_tmp = test(worker.model,args.criterion,worker.testloader)
    acc_test.append(acc_tmp)
    loss_test.append(loss_tmp)
    print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    worker.model = worker.model.to('cpu')
    
end = time.time()#終了時刻

Worker1 accuracy:42.43938280675974  loss:2.178865058081491
Worker2 accuracy:19.243986254295532  loss:3.2454605102539062
Worker3 accuracy:11.363636363636363  loss:3.1809258460998535
Worker4 accuracy:19.696969696969695  loss:3.285339593887329
Worker5 accuracy:25.0  loss:3.129007339477539
Worker6 accuracy:19.047619047619047  loss:3.8749141693115234
Worker7 accuracy:0.0  loss:4.449426174163818
Worker8 accuracy:20.66115702479339  loss:3.0661745071411133
Worker9 accuracy:0.0  loss:4.375886917114258
Worker10 accuracy:15.92741935483871  loss:3.2231366634368896
Worker11 accuracy:24.299065420560748  loss:3.017181158065796
Worker12 accuracy:17.5  loss:3.2829365730285645
Worker13 accuracy:36.31457208943716  loss:2.287992318471273
Worker14 accuracy:15.868263473053892  loss:3.053452253341675
Worker15 accuracy:12.5  loss:4.213351249694824
Worker16 accuracy:37.04752911551778  loss:2.212414264678955
Worker17 accuracy:19.23076923076923  loss:3.3681042194366455
Worker18 accuracy:22.972972972972972  loss:

In [23]:
print('推論時間：{}秒'.format(end-start))#終了時刻-開始時刻でかかった時間

推論時間：10.104694128036499秒


In [24]:
acc_valid_avg = sum(acc_valid)/len(acc_valid)
loss_valid_avg = sum(loss_valid)/len(loss_valid)
print('Validation  loss:{}  accuracy:{}'.format(loss_valid_avg,acc_valid_avg))

acc_test_avg = sum(acc_test)/len(acc_test)
loss_test_avg = sum(loss_test)/len(loss_test)
print('Test  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Validation  loss:3.356172227859497  accuracy:17.844310603575472
Test  loss:3.276311029139019  accuracy:19.708129094217433
